In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import sys
import csv
import datetime
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from scipy.stats import gaussian_kde

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv',index_col='id')
df_test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv', index_col ='id')

In [ ]:
df_train_x = df_train.drop('target', axis=1)
df_train_y = df_train['target']
df_test_x = df_test

In [ ]:
candles = [
    'f0','f2','f4','f9','f12','f16','f19','f20','f23','f24','f27',
    'f28','f30','f31','f32','f33','f35','f39','f42','f44','f46','f48',
    'f49','f51','f52','f53','f56','f58','f59','f60','f61','f62','f63',
    'f64','f68','f69','f72','f73','f75','f76','f78','f79','f81','f83',
    'f84','f87','f88','f89','f90','f92','f93','f94','f95','f98','f99'
]

def candle_log_transform(df_x, candles):
    df_candles_log_transform = df_x[candles]
    df_not_candles = df_x.drop(candles,axis=1)

    mask_neg = (df_candles_log_transform < 0)
    mask_pos = (df_candles_log_transform > 0)

    df_candles_log_transform[mask_neg] = np.log(np.abs(df_candles_log_transform)) * (-1)
    df_candles_log_transform[mask_pos] = np.log(df_candles_log_transform)

    return pd.concat([df_not_candles, df_candles_log_transform],axis=1)

df_train_x = candle_log_transform(df_train_x, candles)
df_test_x = candle_log_transform(df_test_x, candles)

In [ ]:
agg_features = ['sum','mean','std','max','min','kurt']

for ft in agg_features:
    class_method = getattr(pd.DataFrame, ft)
    df_train_x[ft] = class_method(df_train_x, axis=1)
    df_test_x[ft] = class_method(df_test_x, axis=1)

#scaler = StandardScaler()
scaler = QuantileTransformer()
scaler.fit(df_train_x)

df_train_x = pd.DataFrame(scaler.transform(df_train_x),columns=df_train_x.columns)
df_test_x = pd.DataFrame(scaler.transform(df_test_x),columns=df_test_x.columns)

df_train = pd.concat([df_train_x.reset_index(drop=True), df_train_y.reset_index(drop=True)],axis=1)
df_train.head(3)

In [ ]:
corr = df_train.corr()

fig, axes = plt.subplots(figsize=(30, 15))
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, mask=mask, linewidths=.5, cmap='tab20c',vmin=-0.4, vmax=0.4)

plt.show()

In [ ]:
corr_cp = corr['target'].copy()
corr['target_']=corr_cp.abs()
corr_rank = corr.sort_values('target_',ascending=False).iloc[:,-2:]
corr_rank['col']=corr.index.to_series()

fig = plt.figure(figsize=(10,30), facecolor="white")
plt.tick_params(top=True, labeltop=True)
sns.barplot(data=corr_rank.drop('target',axis=0),y='col',x='target')

In [ ]:
# KL divergence
def KLdivergence(p,q,dx):
    KL=np.sum(p * np.log(p/q)) * dx
    return KL

def cal_kld(df, y_name):
    kld_list=[]

    for col in df.drop(y_name,axis=1).columns:
        df_col_0 = df.query('{0} == 0'.format(y_name))[col].to_numpy()
        df_col_1 = df.query('{0} == 1'.format(y_name))[col].to_numpy()

        #print(df_train_col_0)
        limmin = np.percentile(df[col], 0.1)
        limmax = np.percentile(df[col], 99.9)
        ls = np.linspace(limmin, limmax, 100)

        kde_0 = gaussian_kde(df_col_0)
        kde_1 = gaussian_kde(df_col_1)
        kde_value_0 = kde_0(ls)
        kde_value_1 = kde_1(ls)

        kld = KLdivergence(kde_value_0, kde_value_1,dx=(limmax - limmin)/100)

        kld_list.append([col , kld])
    
    return kld_list

kld_list = cal_kld(df_train, 'target')

In [ ]:
df_kld = pd.DataFrame(kld_list, columns=['col','kld'])
df_kld_sorted = df_kld.sort_values('kld', ascending=False)

fig = plt.figure(figsize=(10,30), facecolor="white")
sns.barplot(data=df_kld_sorted,y='col',x='kld')

In [ ]:
features = df_kld_sorted['col'].values

plt.figure()
fig, ax = plt.subplots(figsize=(25,25))

for i,feature in enumerate(features):
    plt.subplot(11, 10,i+1)
    sns.kdeplot(data=df_train,x=df_train[feature],hue='target')
    
    ax = plt.gca()
    ax.set_title(feature, loc='center')
    if i!=0:
        ax.axes.get_legend().remove()
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.ylabel('')

plt.show()